## 2 . Temporal clasification
Let $S$ be a set of training example drawn from a fixed distribution $\mathcal{D_{X\times Z}}$. The input space $(\mathbb{R}^m)^*$ is the set of all sequences of $m$ dimenssional real valued vectors. The target space $\mathcal{Z}=L^*$ is the set of all sequences over the (finite) alphabet $L$ of labels. In general, we refer to elements of $L^*$ as *label sequences* or *labelings*. Each example in $S$ consists of a pair of sequences $(\mathbf{x},\mathbf{z})$. The target sequence $\mathbf{z}=(z_1, z_2, ...,z_U)$ is at most as long as the input sequence $\mathbf{x}=(x_1, x_2, ..., x_T)$, i.e. $U\leq T$. Since the input and target sequences are not generally the same length, there is no *priori* way of aligning them.

The aim is to use $S$ to train a temporal classifier $h: \mathcal{X}\rightarrow\mathcal{Z}$ to classify previously unseen input sequences in a way that minimises some task specifis erroe measure.

### 2.1. Label Error Rate
In this paper, we are interested in the following error measure: given a test set $S' \subset \mathcal{D_{X\times Z}}$ disjoint from $S$, define the *label erroe rate (LER)* of a temporal calssifier $h$ as the normalised edit distance between its classifications and the targets on $S'$, i.e.:

$$
\text{LER}(h,S')=\frac{1}{Z}\sum_{(\mathbf{x}, \mathbf{s})\in S'}{\text{ED}(h(\mathbf{x}))}
\tag{1}$$

where $Z$ is the total number of target labels in $S'$, and $\text{ED}(\mathbf{p}, mathbf{q})$ is the distance between the two sequences $\mathbf{p}$ and $\mathbf{q}$ -- i.e. the minimum number of insertions, substitutions and deletions required to change $\mathbf{p}$ to $\mathbf{q}$.

This is a natural measure for tasks (such as speech or handwriting recognition) where the aim is to minimise the rate of transcription mistakes.

## 3. Connectionist Temporal Classification
This section describes the output representation that allows a recurrent neural network to be used for CTC. The crucial step is to transform the network outputs into a conditional probability distribution over label sequences. The network can then be used a classifier by selecting the most probable labelling for a given input sequence.

### 3.1.  From Network Outputs to Labellings
A CTC network has a softmax output layer with one more unit than there are labels in $L$. The activations of the first $|L|$ units are interpreted as the probabilities of observing the corresponding labels at particular times. The activation of the extra unit is the probability of observing a 'blank', or no label. Together, these outputs define the probabilities of all possible ways of aligning all possible lebel sequences with the input sequence. The total probability of any one label sequence can then be found by summing the probabilities of its different alignments.

More formally, for an input sequence $\mathbf{x}$ of length $T$, define a recurrent neural network with $m$ inputs, $n$ outputs and weights vector #w# as a continuous map $\mathcal{N}_w: (\mathbb{R}^m)^T \rightarrow (\mathbb{R}^n)^T$. Let $\mathbf{y}=\mathcal{N}_w(\mathbf{x})$ be the sequence of network outputs, and denote by $y_k^t$ the activation of output unit $k$ at time $t$. Then $y_k^t$ is interpreted as the probability of observing label $k$ at time $t$, which defines a distribution over the set $L'^T$ of length $T$. sequnces over the alphabet $L'=L\cup\{blank\}$:

$$
p(\pi\mid\mathbf{x}) = \prod_{t=1}^T {y^t_{\pi_t}}, \forall \pi \in L'^T
\tag{2}$$

... Finally we use $\mathcal{B}$ to define the conditional probability of a given labelling $\mathbf{l}\in L^{\leq T}$ as the sum of the probabilities of all the paths corresponding to its:

$$
p(\mathbf{l}\mid\mathbf{x})=\sum_{\pi\in\mathcal{B}^{-1}(\mathbf{l})}{p(\pi\mid\mathbf{x})}
\tag{3}$$

### 3.2. Constructing the Classifier
Given the above formulation, the output of the classifier should be the most probable labelling for the input sequence:

$$
h(\mathbf{x}) = \text{arg}\max_{\mathbf{l}\in L^{\leq T}}p(\mathbf{l\mid x})
$$
...
The first method (best path decoding) is based on the assumption that the most probable path will correspond to the most probable labelling:

$$
h(\mathbf{x})\approx\mathcal{B}(\pi^*) \text{ where } \pi^* = \text{arg}\max_{\pi \in N^t}{p(\pi\mid\mathbf{x})}
\tag{4}$$ ...

## 4. Training the Network
...
### 4.1. The CTC Forward-Bacward Algorithm
$$\displaystyle \alpha_t(s)\overset{\text{def}}{=}\sum_{\pi\in N^T:\ \mathcal{B}(\pi_{1:t})\ =\ \mathbf{l}_{1:s}}\prod_{t'\ =\ 1}^{t}y_{\pi_{t'}}^{t'} \tag{5}$$

This gives us the following rules for initialisation:
$$
\begin{alignat*}{2} 
\alpha_1(1) &= y_b^1\\
\alpha_1(2)&=y^1_{\mathbf{l}_1}\\
\alpha_1(s)&=0, \forall s>2
\end{alignat*}
$$

and recusion:

$$
\alpha_t(s)=\left\{ 
    \begin{matrix} 
        \bar{\alpha}_t(s)y^t_{\mathbf{l}'_s} & \text{if } \mathbf{l}'_s = b \text{ or } \mathbf{l}'_{s-2} = \mathbf{l}'_s\\
        \left(\bar{\alpha}_t(s)+\alpha_{t-1}(s-2)\right)y^t_{\mathbf{l}'_s}&\text{otherwise}
    \end{matrix} 
\right.
\tag{6}$$

where:

$$\bar{\alpha}_t(s)\overset{\text{def}}{=}\alpha_{t-1}(s)+\alpha_{t-1}(s-1)\tag{7}$$


The probability of $\mathbf{l}$ is then the sum of the total probabilities of $\mathbf{l}'$ with and without th final blanh at time $T$
$$p(\mathbf{l\mid x})=\alpha_T(|\mathbf{l}'|)+\alpha_T(|\mathbf{l}'|-1)\tag{8}$$

Similarly, the backward variables $\beta_t(s)$ are defined as the total probability of $\mathbf{l}_{s:|\mathbf{l}|}$ at time $t$:

$$\displaystyle
\beta_t(s)\ \overset{\text{def}}{=}\ \sum_{\pi\in N^T\ :\ \mathcal{B}(\pi_{t:T})\ =\ \mathbf{l}_{s:|\mathbf{l}|}}{\prod_{t'\ =\ t}^{T}{y^{t'}_{\pi_{t'}}}}
\tag{9}$$

$$
\begin{alignat*}{2}
\beta_T(|\mathbf{l'}|) &= y_b^T\\
\beta_T(|\mathbf{l'}|-1) &= y_{\mathbf{l_{|l|}}}^T\\
\beta_T(s) & =0, \ \forall s<|\mathbf{l}|-1
\end{alignat*}
$$

$$
\beta(s)=\left\{
\begin{matrix}
\bar{\beta}_t(s)y^t_{\mathbf{l}_s'} & \text{if}\ \ \mathbf{l}_s' = b \text{ or } \mathbf{l}_{s+2}'=\mathbf{l}_s'\\
\left(\bar{\beta}_t(s)+\beta_{t+1}(s+2)\right)y^t_{\mathbf{l}_s'}&\text{otherwise}
\end{matrix}
\right.
\tag{10}$$

where

$$\bar{\beta}(s)\overset{\text{def}}{\ =\ }\beta_{t+1}(s)+\beta_{t+1}(s+1)\tag{11}$$

Note that $\beta_{t}(s)=0 \forall s>2t$ (the unconnected circles in the bottom left of figure 3) and $\forall s>\mathbf{|l'|}$

In practice, the above recursions will soon lead to underflows on any difital computer. One way of avoiding this is to rescale the forward and backward variables. If we define 

$$
C_t\overset{\text{def}}{=}\sum_s{\alpha_t(s)}\quad\quad\quad \hat{\alpha}(s)\overset{\text{def}}{=}\frac{\alpha_t(s)}{C_t}
$$

and substitue $\alpha$ for $\hat{\alpha}$ on the RHS of (6) and (7), the forward variables will remain in computational range. Similarly, for the backward variables we define:

$$
D_t\overset{\text{def}}{=}\sum_s{\beta_t(s)}\quad\quad\quad \hat{\beta}(s)\overset{\text{def}}{=}\frac{\beta_t(s)}{D_t}
$$

and substitue $\beta$ for $\hat{\beta}$ on the RHS of (10) and (11).

To evaluate the maximum likelihood error, we need the natural logs of the target labelling probabilities. With the rescaled variables these have a particularly simple form:

$$
\ln\left(p(\mathbf{l\mid x})\right)=\sum_{t=1}^{T}{\ln(C_t)}
$$

### 4.2 Maximum Likelihood Training
The aim of maximum likelihood training is to simultaneously maximize the log probabilities of all the correct classifications in the training ser. In our case, this means minimising the following objective function:

$$
O^{ML}(S,\mathcal{N}_w)=-\sum_{\mathbf{(x,z)}\in S}{\ln(p(\mathbf{z\mid x}))}
\tag{12}$$

To train the network with gradient descent, we need to differentiate (12) with respect to the network outputs. Since the trining examples are independent we can consider them separately:

$$
\frac{\partial}{\partial y^t_k}{O^{ML}(\{(\mathbf{x,z})\},\mathcal{N}_w)}=-\frac{\partial}{\partial y^t_k}{\ln(p(\mathbf{z\mid x}))} 
\tag{13}$$
...
More precisely, from (5) and (6) we have:

$$\displaystyle
\alpha_t(s)\beta_t(s)=\sum_{\pi\ \in\ \mathcal{B}^{-1} (\mathbf{l}):\ \pi_t\ =\ \mathbf{l}'_s  }y^t_{\mathbf{l'}_s}\prod_{t=1}^T{y^t}_{\pi_t}
$$

Rearranging and substituting in from (2) gives:

$$
\frac{\alpha_t(s)\beta_t(s)}{y^t_{\mathbf{l}'_s}}=\sum_{\pi\ \in\ \mathcal{B}^{-1} (\mathbf{l}):\ \pi_t\ =\ \mathbf{l}'_s}p(\pi\mid\mathbf{x})
$$

...

From (3) we can see that this is the portion of the total probability $p(\mathbf{l\mid x})$ due to those paths going through $\mathbf{l}'_s$ at time $t$. For any $t$ we can therefore sum voer sll $s$ to get:

$$
p(\mathbf{l\mid x})=\sum_{s=1}^{\mathbf{l}'}\frac{\alpha_t(s)\beta_t(s)}{y^t_{\mathbf{l}'_s}}
\tag{14}$$

...

We the differentiate (14) to get:

$$
\frac{\partial}{\partial y^t_k}p(\mathbf{l\mid x})=\frac{1}{y^{t2}_k}\sum_{s\in \text{label}(\mathbf{l},k)} \alpha_t(s)\beta_t(s)
\tag{15}$$

Observing that:

$$
\frac{\partial}{\partial y^t_l}\ln(p(\mathbf{l\mid x}))=\frac{1}{p(\mathbf{l\mid x})}\frac{\partial}{\partial y^t_k}p(\mathbf{l\mid x})
$$

we can set $\mathbf{l=z}$ and subtitue (8) and (15) into (13) to differentitate the objective function.

...
If the rescaling of section 4.1 is used, we have:

$$
\frac{\partial}{\partial y^t_k}{O^{ML}(\{(\mathbf{x,z}),\mathcal{N}_w\})}=y^t_k- \frac{1}{y^t_kZ_t}\sum_{s\ \in\ \text{label}(\mathbf{z},k)}{\hat{\alpha}_t(s)\hat{\beta}_t(s)}
\tag{16}$$

where

$$
Z_f\overset{\text{def}}{\ =\ }\sum_{s=1}^{|\mathbf{l}'|}{\frac{\hat{\alpha}_t(s)\hat{\beta}_t(s)}{y^t_{\mathbf{l}'_s}}}
$$

Equation (16) is the 'error signal' received by the network during training (figure 4).
